In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_tasks = pd.read_csv('./data/mysql_table_tasks.csv')
data_reg_reg = pd.read_csv('./data/mysql_table_regular_regular_type.csv')
data_reg_types = pd.read_csv('./data/mysql_table_regular_types.csv')
data_regs = pd.read_csv('./data/mysql_table_regulars.csv')

data_tasks = pd.read_csv('./data/mysql_table_tasks.csv')
data_tasks_stages = pd.read_csv('./data/mysql_table_task_stages.csv')
data_tasks_transitions = pd.read_csv('./data/mysql_table_task_transitions.csv')
data_tasks_user = pd.read_csv('./data/mysql_table_task_user.csv')

In [ ]:
data_tasks.taskable_type.unique()

In [ ]:
task_reg_table = data_tasks[data_tasks['taskable_type'] == 'App\\Models\\ServiceDesk\\Regular']

In [ ]:
task_reg_table

In [ ]:
data_regs_merge = task_reg_table.merge(data_reg_reg, left_on='taskable_id', right_on='regular_id')

In [ ]:
data_regs_merge

In [ ]:
data_reg_reg

In [ ]:
data_reg_types['account_id']

# Считаем среднее квадратичное отклонение по времени

In [ ]:
transitions_success = data_tasks_transitions[data_tasks_transitions['task_stage_id'] == 3]

In [ ]:
task_success_merged = task_reg_table.merge(transitions_success, left_on='taskable_id', right_on='task_id')

In [ ]:
task_success_merged

In [ ]:
task_success_merged['delta_time'] = pd.to_datetime(task_success_merged['transitioned_at']) - pd.to_datetime(task_success_merged['deadline_at'])

In [ ]:
task_success_merged['delta_time_days'] = task_success_merged['delta_time'].dt.total_seconds() /60 /60 /24

In [ ]:
np.mean(task_success_merged[task_success_merged['delta_time_days'] < 0]['delta_time_days'])

# Для BACKEND!!!!!!!!!!!!

In [ ]:
#насколько в среднем просрачиваются регламентные работы
mean_dead = [np.mean(task_success_merged[task_success_merged['delta_time_days'] < 0]['delta_time_days'][data_tasks['account_id'] == i]) for i in (1,2)]

#по всем дедлайная медианное значение
mean_all = [np.mean(task_success_merged['delta_time_days'][data_tasks['account_id'] == i]) for i in (1,2)]

In [ ]:
mean_dead

In [ ]:
# matplotlib histogram
plt.figure(figsize=(8, 4))
plt.hist(task_success_merged['delta_time_days'], color = 'blue', edgecolor = 'black',
         bins = int(180/5))

# seaborn histogram
sns.distplot(task_success_merged['delta_time_days'], hist=True, kde=False, 
             bins=int(180/5), color = 'blue',
             hist_kws={'edgecolor':'black'})
# Add labels
plt.title('График отклонений регламентных работ от дедлайнов')
plt.xlabel('Количество дней')
plt.ylabel('Частота')

# plt.legend()
plt.tight_layout()
plt.savefig('./results/reglaments_chast.jpg', dpi=1200)

# plt.show()